In [28]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc,stats
from PIL import Image, ImageDraw

results_file = './results/visgeno_attbilstm_strong_iter_360000_tst.json'
%matplotlib inline

In [2]:
# all boxes are [xmin, ymin, xmax, ymax] format, 0-indexed, including xmax and ymax
def compute_bbox_iou(bboxes, target):
    if isinstance(bboxes, list):
        bboxes = np.array(bboxes)
    bboxes = bboxes.reshape((-1, 4))

    if isinstance(target, list):
        target = np.array(target)
    target = target.reshape((-1, 4))

    A_bboxes = (bboxes[..., 2]-bboxes[..., 0]+1) * (bboxes[..., 3]-bboxes[..., 1]+1)
    A_target = (target[..., 2]-target[..., 0]+1) * (target[..., 3]-target[..., 1]+1)
    assert(np.all(A_bboxes >= 0))
    assert(np.all(A_target >= 0))
    I_x1 = np.maximum(bboxes[..., 0], target[..., 0])
    I_y1 = np.maximum(bboxes[..., 1], target[..., 1])
    I_x2 = np.minimum(bboxes[..., 2], target[..., 2])
    I_y2 = np.minimum(bboxes[..., 3], target[..., 3])
    A_I = np.maximum(I_x2 - I_x1 + 1, 0) * np.maximum(I_y2 - I_y1 + 1, 0)
    IoUs = A_I / (A_bboxes + A_target - A_I)
    assert(np.all(0 <= IoUs) and np.all(IoUs <= 1))
    return IoUs


def evaluate_bboxes(results, thresh_k=1):
    for res in results:
        pred_bboxes = res['predicted_bounding_boxes']
        pred_bboxes = pred_bboxes[:thresh_k]
        ground_truth = res['ground_truth']
        
        ious = compute_bbox_iou(pred_bboxes, ground_truth)
        res['iou'] = np.max(ious)
        res['iou_max_idx'] = np.argmax(ious)

In [3]:
def draw_bbox(ax, bbox, edge_color='red', line_width=3):
    """Draw one bounding box on a matplotlib axis object (ax)."""
    import matplotlib.patches as mpatches
    
    bbox_plot = mpatches.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
        fill=False, edgecolor=edge_color, linewidth=line_width)
    ax.add_patch(bbox_plot)

In [4]:
def visualize_bounding_boxes(pred_sample):
    iou = pred_sample['iou']
    
    bbox_pred_top = pred_sample['predicted_bounding_boxes'][pred_sample['iou_max_idx']]
    gt = pred_sample['ground_truth'][0]
    print(gt)
    I = misc.imread(pred_sample['image_path'])
    print(I.shape)
    ax = plt.imshow(I)
    ax = plt.axis('off')
    ax = plt.title('IoU: %.3f, green bbox: GT, red bbox: predicted' % iou)
    draw_bbox(plt.gca(), gt, edge_color='green')
    draw_bbox(plt.gca(), bbox_pred_top, edge_color='red')
    
    
# I = misc.imread('/home/dwright/repos/papers/Hu et al 2017/exp-visgeno-rel/visgeno-dataset/VG_100K_2/63.jpg')
# ax = plt.imshow(I)
# ax = plt.axis('off')
# draw_bbox(plt.gca(), [ 295.,  309.,  339.,  484.], edge_color='green')

In [5]:
from matplotlib.colors import LinearSegmentedColormap
colors = [(1, 1, 1), (1, 0, 0)]
cm = LinearSegmentedColormap.from_list(
        'text_cmap', colors)
def makeHeatmap(attention, words, ax, title=""):
    plt.figure(figsize=(18, 1.5))
    heatmap = plt.pcolor(attention, cmap=cm)
    i = 0
    for x in range(len(attention[0])):
        plt.text(x + 0.5, 0.5, words[i], horizontalalignment='center', verticalalignment='center')
        i += 1
        
    plt.gca().get_xaxis().set_visible(False)
    plt.title(title)

In [6]:
from matplotlib.backends.backend_pdf import PdfPages

def multipage(filename, figs=None, dpi=200):
    pp = PdfPages(filename)
    if figs is None:
        figs = [plt.figure(n) for n in plt.get_fignums()]
    for fig in figs:
        fig.savefig(pp, format='pdf')
    pp.close()

In [7]:
import re

SENTENCE_SPLIT_REGEX = re.compile(r'(\W+)')
def split(sentence):
    if isinstance(sentence, bytes):
        sentence = sentence.decode()
    words = SENTENCE_SPLIT_REGEX.split(sentence.strip())
    words = [w.lower() for w in words if len(w.strip()) > 0]
    # remove .
    if len(words) > 0 and (words[-1] == '.' or words[-1] == '?'):
        words = words[:-1]
    return words

In [8]:
with open(results_file) as f:
    results = json.load(f)

In [9]:
evaluate_bboxes(results)

In [10]:
def filterValidBboxes(samples):
    valid = []
    for s in samples:
        I = misc.imread(s['image_path'])
        r1 = I.shape
        r2 = s['ground_truth'][0]
        r3 = s['predicted_bounding_boxes'][s['iou_max_idx']]
        if 0 <= r2[0] < r2[2] <= r1[0] and 0 <= r2[1] < r2[3] <= r1[1] and 0 <= r3[0] < r3[2] <= r1[0] and 0 <= r3[1] < r3[3] <= r1[1]:
            valid.append(s)
    return valid 

In [11]:
bad_results = [res for res in results if res['iou'] > 0.5]
bad_results = filterValidBboxes(bad_results)

In [19]:
good_results = [res for res in results if res['iou'] > 0.5]
# Randomly sample 100 bad examples
for i in range(100):
    sample = good_results[i]
    visualize_bounding_boxes(sample)
     
    ref = sample['refexp']
    ref = split(ref)
    sub = [s[0] for s in sample['obj1_prob']][-len(ref):]
    makeHeatmap([sub], ref, plt.gca(), title='Subject attn')
    rel = [s[0] for s in sample['rel_prob']][-len(ref):]
    makeHeatmap([rel], ref, plt.gca(), title='Relation attn')
    obj = [s[0] for s in sample['obj2_prob']][-len(ref):]
    makeHeatmap([obj], ref, plt.gca(), title='Object attn')
    multipage('/hdd/dustin/data/hu_et_al_eval/visgeno_strong_positive_samples/%.3d.pdf' % i)
    #multipage('/hdd/dustin/data/hu_et_al_eval/negative_samples/test.pdf')
    plt.close('all')

[299.0, 305.0, 333.0, 477.0]
(600, 800, 3)
[301.0, 301.0, 340.0, 497.0]
(600, 800, 3)
[26.0, 173.0, 262.0, 340.0]
(600, 800, 3)
[303.0, 305.0, 331.0, 489.0]
(600, 800, 3)
[9.0, 3.0, 794.0, 222.0]
(600, 800, 3)
[64.0, 360.0, 114.0, 559.0]
(600, 800, 3)
[26.0, 173.0, 262.0, 340.0]
(600, 800, 3)
[64.0, 360.0, 114.0, 559.0]
(600, 800, 3)
[26.0, 173.0, 262.0, 340.0]
(600, 800, 3)
[26.0, 173.0, 262.0, 340.0]
(600, 800, 3)
[58.0, 356.0, 115.0, 558.0]
(600, 800, 3)
[295.0, 309.0, 339.0, 484.0]
(600, 800, 3)
[3.0, 142.0, 795.0, 593.0]
(600, 800, 3)
[3.0, 142.0, 795.0, 593.0]
(600, 800, 3)
[0.0, 6.0, 780.0, 122.0]
(600, 800, 3)
[41.02564239501953, 32.05128479003906, 67.94872283935547, 73.0769271850586]
(502, 468, 3)
[41.02564239501953, 32.05128479003906, 67.94872283935547, 73.0769271850586]
(502, 468, 3)
[41.02564239501953, 32.05128479003906, 67.94872283935547, 73.0769271850586]
(502, 468, 3)
[41.02564239501953, 32.05128479003906, 67.94872283935547, 73.0769271850586]
(502, 468, 3)
[518.0, 366.0,

In [21]:
bad_results = [res for res in results if res['iou'] < 0.05]
# Randomly sample 100 bad examples
for i in range(100):
    sample = bad_results[np.random.randint(0, len(bad_results))]
    visualize_bounding_boxes(sample)
     
    ref = sample['refexp']
    ref = split(ref)
    sub = [s[0] for s in sample['obj1_prob']][-len(ref):]
    makeHeatmap([sub], ref, plt.gca(), title='Subject attn')
    rel = [s[0] for s in sample['rel_prob']][-len(ref):]
    makeHeatmap([rel], ref, plt.gca(), title='Relation attn')
    obj = [s[0] for s in sample['obj2_prob']][-len(ref):]
    makeHeatmap([obj], ref, plt.gca(), title='Object attn')
    multipage('/hdd/dustin/data/hu_et_al_eval/visgeno_strong_negative_samples/%.3d.pdf' % i)
    #multipage('/hdd/dustin/data/hu_et_al_eval/negative_samples/test.pdf')
    plt.close('all')

[294.3999938964844, 267.20001220703125, 480.0, 489.6000061035156]
(375, 500, 3)
[395.20001220703125, 1.600000023841858, 540.7999877929688, 108.80000305175781]
(375, 500, 3)
[539.2000122070312, 411.20001220703125, 584.0, 454.3999938964844]
(375, 500, 3)
[160.0, 168.0, 179.1999969482422, 217.60000610351562]
(500, 375, 3)
[470.270263671875, 336.93695068359375, 495.4954833984375, 418.01800537109375]
(333, 500, 3)
[259.45947265625, 491.89190673828125, 290.090087890625, 515.3153076171875]
(333, 500, 3)
[57.26141357421875, 148.13278198242188, 125.72614288330078, 296.26556396484375]
(482, 500, 3)
[744.5783081054688, 381.3253173828125, 771.686767578125, 397.5903625488281]
(332, 500, 3)
[392.79278564453125, 180.18017578125, 423.4234313964844, 203.60360717773438]
(333, 500, 3)
[117.10526275634766, 192.10525512695312, 131.57894897460938, 205.26315307617188]
(456, 500, 3)
[280.0, 276.6666564941406, 420.0, 331.6666564941406]
(360, 500, 3)
[184.8000030517578, 445.20001220703125, 198.00001525878906, 4

In [65]:
for res in results:
    res['iou'] = float(res['iou'])
    res['iou_max_idx'] = int(res['iou_max_idx'])
with open('./results/visgeno_res_iou_backup.json', 'w') as f:
    json.dump(results, f)

In [24]:
print("p@1, IoU > 0.5: " + str(len(good_results) / float(len(results))))

p@1, IoU > 0.5: 0.500460393271


In [30]:
sr_kl = []
rs_kl = []
so_kl = []
os_kl = []
ro_kl = []
or_kl = []

for res in results:
    ref = res['refexp']
    ref = split(ref)
    sub = [s[0] for s in res['obj1_prob']][-len(ref):]
    rel = [s[0] for s in res['rel_prob']][-len(ref):]
    obj = [s[0] for s in res['obj2_prob']][-len(ref):]
    
    sr_kl.append(stats.entropy(sub, rel))
    rs_kl.append(stats.entropy(rel, sub))
    
    so_kl.append(stats.entropy(sub, obj))
    os_kl.append(stats.entropy(obj, sub))
    
    ro_kl.append(stats.entropy(rel, obj))
    or_kl.append(stats.entropy(obj, rel))
    
print('Avg K(s||r): ', np.mean(sr_kl))
print('Avg K(r||s): ', np.mean(rs_kl))

print('Avg K(s||o): ', np.mean(so_kl))
print('Avg K(o||s): ', np.mean(os_kl))

print('Avg K(r||o): ', np.mean(ro_kl))
print('Avg K(o||r): ', np.mean(or_kl))

('Avg K(s||r): ', 0.60251183270256126)
('Avg K(r||s): ', 0.51793136047912214)
('Avg K(s||o): ', 0.53593227345041183)
('Avg K(o||s): ', 0.45483972083857094)
('Avg K(r||o): ', 0.11976711577362126)
('Avg K(o||r): ', 0.1303244905852381)


0.023699664429530202